## 6-7. 프로젝트: 멋진 작사가 만들기

### [루브릭]
평가문항	상세기준
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?  
  -텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?  
      
      
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?  
  -특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?  
     
     
3. 텍스트 생성모델이 안정적으로 학습되었는가?  
  -텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?  
    
### [학습 과정]
1. 데이터 다운로드
2. 데이터 읽어오기
3. 데이터 정제
4. 평가 데이터셋 분리
5. 인공지능 만들기

### [결과 및 회고]

### 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/E06_LyricistAI/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


### 데이터 정제하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Verse 1]
They come from everywhere
A longing to be free
They come to join us here
From sea to shining sea And they all have a dream
As people always will
To be safe and warm
In that shining city on the hill Some wanna slam the door
Instead of opening the gate
Aw, let's turn this thing around


 평가 데이터셋 분리

인공지능 만들기